In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from tqdm import tqdm
tqdm.pandas()
import time 
from vllm import LLM, SamplingParams

# vllm

In [2]:
config = dict(
    max_new_tokens=512, 
    # do_sample=True, 
    # num_beams=1, 
    temperature=0.25, 
    top_k=50, 
    top_p=0.98, 
    stop_token_ids =[79097]
)

model_name = "Vikhrmodels/Vikhr-7B-instruct_0.4"

llm = LLM(model=model_name, dtype="bfloat16", tensor_parallel_size=2)

sampling_params = SamplingParams(
    max_tokens=config["max_new_tokens"],
    temperature=config["temperature"],
    top_k=config["top_k"],
    top_p=config["top_p"],
    stop_token_ids =config["stop_token_ids"],
    skip_special_tokens = False #ОЧЕНЬ ВАЖНО! иначе не будет остановки после стоп токена 



)

print("Stop token IDs:", sampling_params.stop_token_ids)

INFO 01-19 20:16:52 config.py:510] This model supports multiple tasks: {'generate', 'embed', 'classify', 'reward', 'score'}. Defaulting to 'generate'.
INFO 01-19 20:16:52 config.py:1310] Defaulting to use mp for distributed inference
INFO 01-19 20:16:52 llm_engine.py:234] Initializing an LLM engine (v0.6.6.post1) with config: model='Vikhrmodels/Vikhr-7B-instruct_0.4', speculative_config=None, tokenizer='Vikhrmodels/Vikhr-7B-instruct_0.4', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 01-19 20:17:05 model_runner.py:1099] Loading model weights took 7.1138 GB
(VllmWorkerProcess pid=176278) INFO 01-19 20:17:05 model_runner.py:1099] Loading model weights took 7.1138 GB
(VllmWorkerProcess pid=176278) INFO 01-19 20:17:11 worker.py:241] Memory profiling takes 5.07 seconds
(VllmWorkerProcess pid=176278) INFO 01-19 20:17:11 worker.py:241] the current vLLM instance can use total_gpu_memory (39.38GiB) x gpu_memory_utilization (0.90) = 35.44GiB
(VllmWorkerProcess pid=176278) INFO 01-19 20:17:11 worker.py:241] model weights take 7.11GiB; non_torch_memory takes 0.29GiB; PyTorch activation peak memory takes 2.07GiB; the rest of the memory reserved for KV Cache is 25.97GiB.
INFO 01-19 20:17:11 worker.py:241] Memory profiling takes 5.16 seconds
INFO 01-19 20:17:11 worker.py:241] the current vLLM instance can use total_gpu_memory (39.38GiB) x gpu_memory_utilization (0.90) = 35.44GiB
INFO 01-19 20:17:11 worker.py:241] model weights take 7.11GiB; non_torch_memory takes 0.29GiB; Py

Capturing CUDA graph shapes: 100%|██████████████| 35/35 [00:16<00:00,  2.07it/s]

INFO 01-19 20:17:29 model_runner.py:1535] Graph capturing finished in 17 secs, took 0.45 GiB
(VllmWorkerProcess pid=176278) INFO 01-19 20:17:29 model_runner.py:1535] Graph capturing finished in 17 secs, took 0.45 GiB
INFO 01-19 20:17:29 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 23.94 seconds


Stop token IDs: [79097]
(VllmWorkerProcess pid=176278) INFO 01-19 20:22:02 multiproc_worker_utils.py:247] Worker exiting


### output

In [3]:
DEFAULT_SYSTEM_PROMPT = "Ты получишь тексты отзывов покупателей об одном продукте. Выдели кратко не больше 5 основных моментов, которые отмечают покупатели."


conversation = [
    {
        "role": "system",
        "content": DEFAULT_SYSTEM_PROMPT
    },
    {
        "role": "user",
        "content": "Огромные и не очень вкусные Замечательные!! Мне очень понравились яблоки этого сорта. Сочные, сладкие! Вата, а не яблоки Сочные, вкусные Привезли очень крупные яблоки! На вкус ароматные и сладкие. Для тех кто любит мягкие. Сочные сладкие крепкие Яблоки просто восторг!- не ожидала! Сочные, ароматные, не жесткие- даже кожа хорошо жуётся! Сладкие. Брала со скидкой. Закажу ещё на завтра. Не уверена что привезут такие же- иначе бы кг 3 заказала. Привезли в заказе ужасные яблоки!!Есть такие невозможно (( Небольшие сочные хрустящие яблочки, блеск Крупненькие оказались, ожидала мелкие яблочки. Красивее, чем на фото.) Мне для пирога. Спасибо. Впервые в продаже такие красивые сезонные яблоки. И вкусные. Яблоки вкусные, но очень большие. Хотелось бы поменьше размером. Крупные. Мало косточек. Кисло сладкие. Мякоть мягкая и сочная. Кожица не толстая. Очень вкусные! Ужасные яблоки, кожура потемневшая, внутри ватные Очень вкусные и сладкие яблоки. Сочные. Приятные яблочки. Сочные, в меру сладкие. Кожица несколько плотновата. Хорошие крупные яблоки. Вкусные. Очень ароматные и сладкие! Те, что темно-красные даже очень сладкие! Хорошие мелкие сезонные яблоки. Адекватная цена. Вкусные яблоки, душистые"
    },
]
outputs = llm.chat(conversation,
                   sampling_params=sampling_params,
                   use_tqdm=False)

outputs[0].outputs[0].text.strip()

INFO 01-19 20:17:47 chat_utils.py:333] Detected the chat template content format to be 'string'. You can set `--chat-template-content-format` to override this.


'1. Покупатели ценят яблоки за их сочность, сладость и хрустящую текстуру.\n2. Некоторые покупатели предпочитают яблоки поменьше, в то время как другие ценят крупные и ароматные.\n3. Кожура яблок описана как блестящая, красивая и не слишком толстая.\n4. Некоторые покупатели отмечают, что яблоки могут быть немного ватными, особенно у тех, что крупнее.\n5. Цены считаются разумными, и покупатели готовы заказывать снова, несмотря на некоторые колебания по поводу качества.'

In [4]:
print(outputs[0].outputs[0].text.strip())

1. Покупатели ценят яблоки за их сочность, сладость и хрустящую текстуру.
2. Некоторые покупатели предпочитают яблоки поменьше, в то время как другие ценят крупные и ароматные.
3. Кожура яблок описана как блестящая, красивая и не слишком толстая.
4. Некоторые покупатели отмечают, что яблоки могут быть немного ватными, особенно у тех, что крупнее.
5. Цены считаются разумными, и покупатели готовы заказывать снова, несмотря на некоторые колебания по поводу качества.


# vllm server

In [ ]:
# поднимаем сервер в терминале, все параметры как у модели указываем 
# python -m vllm.entrypoints.openai.api_server \
#     --host 0.0.0.0 --port 8080 \
#     --model Vikhrmodels/Vikhr-7B-instruct_0.4 \
#     --dtype=bfloat16 \
#     --tensor_parallel_size=2

In [12]:
%%time
from openai import OpenAI

openai_api_key = "EMPTY"
openai_api_base = "http://0.0.0.0:8080/v1" 

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)
DEFAULT_SYSTEM_PROMPT = "Ты получишь тексты отзывов покупателей об одном продукте. Выдели кратко не больше 5 основных моментов, которые отмечают покупатели."

chat_response = client.chat.completions.create(
    model="Vikhrmodels/Vikhr-7B-instruct_0.4",
    max_tokens=512,
    temperature=0.25, 
    # top_k=50,  #нет такого параметра 
    top_p=0.98, 
    stop=["</s>"], #это вместо stop_token_ids и eos_token_id
    messages=[
    {
        "role": "system",
        "content": DEFAULT_SYSTEM_PROMPT
    },
    {
        "role": "user",
        "content": "Огромные и не очень вкусные Замечательные!! Мне очень понравились яблоки этого сорта. Сочные, сладкие! Вата, а не яблоки Сочные, вкусные Привезли очень крупные яблоки! На вкус ароматные и сладкие. Для тех кто любит мягкие. Сочные сладкие крепкие Яблоки просто восторг!- не ожидала! Сочные, ароматные, не жесткие- даже кожа хорошо жуётся! Сладкие. Брала со скидкой. Закажу ещё на завтра. Не уверена что привезут такие же- иначе бы кг 3 заказала. Привезли в заказе ужасные яблоки!!Есть такие невозможно (( Небольшие сочные хрустящие яблочки, блеск Крупненькие оказались, ожидала мелкие яблочки. Красивее, чем на фото.) Мне для пирога. Спасибо. Впервые в продаже такие красивые сезонные яблоки. И вкусные. Яблоки вкусные, но очень большие. Хотелось бы поменьше размером. Крупные. Мало косточек. Кисло сладкие. Мякоть мягкая и сочная. Кожица не толстая. Очень вкусные! Ужасные яблоки, кожура потемневшая, внутри ватные Очень вкусные и сладкие яблоки. Сочные. Приятные яблочки. Сочные, в меру сладкие. Кожица несколько плотновата. Хорошие крупные яблоки. Вкусные. Очень ароматные и сладкие! Те, что темно-красные даже очень сладкие! Хорошие мелкие сезонные яблоки. Адекватная цена. Вкусные яблоки, душистые"
    },
]
)
print(chat_response.choices[0].message.content)

1. Покупатели ценят яблоки за их сочность, сладость и хрустящую текстуру.
2. Некоторые покупатели предпочитают яблоки поменьше, в то время как другие ценят размер и качество крупных яблок.
3. Яблоки описываются как ароматные, сладкие и вкусные, с некоторыми замечательными вкусами.
4. Кожица яблок описывается как несколько толстая, но все равно приятная на вкус.
5. Покупатели ценят сезонные яблоки и считают, что цена разумная. 

CPU times: user 52.6 ms, sys: 37 μs, total: 52.6 ms
Wall time: 1.28 s


#  vllm LORA

также поднимаем сервер в терминале, все параметры как у модели указываем плюс для LORA --enable-lora, max-lora-rank и --lora-modules 

In [30]:
#складываем модель на хф - ту, которую мы ранее дообучили 
from huggingface_hub import HfApi, upload_folder

api = HfApi()
upload_folder(
    folder_path="/home/d-mishina/comments/attributes/Vikhr-7B-instruct_0.4_lora_r32_medium_prompt",
    repo_id="Daria/Vikhr-7B-instruct_0.4_lora_r32_medium_prompt",
    repo_type="model"
)

/home/d-mishina/vllm/new_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
adapter_model.safetensors: 100%|███████████| 75.5M/75.5M [00:16<00:00, 4.70MB/s]


CommitInfo(commit_url='https://huggingface.co/Daria/Vikhr-7B-instruct_0.4_lora_r32_medium_prompt/commit/5344ed3df89afdb82c450eba4b29b5b9dc84885c', commit_message='Upload folder using huggingface_hub', commit_description='', oid='5344ed3df89afdb82c450eba4b29b5b9dc84885c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Daria/Vikhr-7B-instruct_0.4_lora_r32_medium_prompt', endpoint='https://huggingface.co', repo_type='model', repo_id='Daria/Vikhr-7B-instruct_0.4_lora_r32_medium_prompt'), pr_revision=None, pr_num=None)

__вот так загружаем адаптер с хф__

python -m vllm.entrypoints.openai.api_server \
  --host 0.0.0.0 --port 8080 \
  --model Vikhrmodels/Vikhr-7B-instruct_0.4 \
  --dtype bfloat16 \
  --tensor_parallel_size 2 \
  --enable-lora \
  --max-lora-rank 32 \
  --lora-modules '{
      "name": "Vikhr-7B-instruct_0.4_lora_r32_medium_prompt",
      "path": "Daria/Vikhr-7B-instruct_0.4_lora_r32_medium_prompt",
      "base_model_name": "Vikhrmodels/Vikhr-7B-instruct_0.4"
  }'


__вот так загружаем черезе ДОКЕР, когда адаптер с хф__

docker run --runtime nvidia --gpus all \
-v ~/.cache/huggingface:/root/.cache/huggingface \
-p 8000:8000 \
--ipc=host \
vllm/vllm-openai:latest \
--model Vikhrmodels/Vikhr-7B-instruct_0.4 \
--dtype=bfloat16 \
--tensor_parallel_size=2  \
--enable-lora \
--max-lora-rank=32 \
--lora-modules='{
    "name": "Vikhr-7B-instruct_0.4_lora_r32_medium_prompt",
    "path": "Daria/Vikhr-7B-instruct_0.4_lora_r32_medium_prompt",
    "base_model_name": "Vikhrmodels/Vikhr-7B-instruct_0.4"
}'


__вот так загружаем адаптер с локалки__

python -m vllm.entrypoints.openai.api_server \
  --host 0.0.0.0 --port 8080 \
  --model Vikhrmodels/Vikhr-7B-instruct_0.4 \
  --dtype bfloat16 \
  --tensor_parallel_size 2 \
  --enable-lora \
  --max-lora-rank 32 \
  --lora-modules '{
      "name": "Vikhr-7B-instruct_0.4_lora_r32_medium_prompt",
      "path": "/home/d-mishina/comments/attributes/Vikhr-7B-instruct_0.4_lora_r32_medium_prompt",
      "base_model_name": "Vikhrmodels/Vikhr-7B-instruct_0.4"
  }'

__вот так загружаем черезе ДОКЕР, когда адаптер с локалки__

docker run --runtime nvidia --gpus all \
-v ~/.cache/huggingface:/root/.cache/huggingface \
-p 8000:8000 \
--ipc=host \
vllm/vllm-openai:latest \
--model Vikhrmodels/Vikhr-7B-instruct_0.4 \
--dtype=bfloat16 \
--tensor_parallel_size=2  \
--enable-lora \
--max-lora-rank=32 \
  --lora-modules='{
      "name": "Vikhr-7B-instruct_0.4_lora_r32_medium_prompt",
      "path": "/home/d-mishina/comments/attributes/Vikhr-7B-instruct_0.4_lora_r32_medium_prompt",
      "base_model_name": "Vikhrmodels/Vikhr-7B-instruct_0.4"
  }'

__проверяем, что запустилось в соседнем терминале - должны быть 2 модели__
__не забываем менять порты - если БЕЗ докера, то 8080, если С докером - 8000__

curl localhost:8080/v1/models  | jq .
или 
curl 0.0.0.0:8080/v1/models  | jq .

должно вылезти что-то подобное 


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1080  100  1080    0     0   886k      0 --:--:-- --:--:-- --:--:-- 1054k
{
  "object": "list",
  "data": [
    {
      "id": "Vikhrmodels/Vikhr-7B-instruct_0.4",
      "object": "model",
      "created": 1742066626,
      "owned_by": "vllm",
      "root": "Vikhrmodels/Vikhr-7B-instruct_0.4",
      "parent": null,
      "max_model_len": 32768,
      "permission": [
        {
          "id": "modelperm-f9825ff344e04f73add92ea507b65c00",
          "object": "model_permission",
          "created": 1742066626,
          "allow_create_engine": false,
          "allow_sampling": true,
          "allow_logprobs": true,
          "allow_search_indices": false,
          "allow_view": true,
          "allow_fine_tuning": false,
          "organization": "*",
          "group": null,
          "is_blocking": false
        }
      ]
    },
    {
      "id": "Vikhr-7B-instruct_0.4_lora_r32_medium_prompt",
      "object": "model",
      "created": 1742066626,
      "owned_by": "vllm",
      "root": "comments/attributes/Vikhr-7B-instruct_0.4_lora_r32_medium_prompt",
      "parent": "Vikhrmodels/Vikhr-7B-instruct_0.4",
      "max_model_len": null,
      "permission": [
        {
          "id": "modelperm-3529ac63edb546cca034d75aa80f284d",
          "object": "model_permission",
          "created": 1742066626,
          "allow_create_engine": false,
          "allow_sampling": true,
          "allow_logprobs": true,
          "allow_search_indices": false,
          "allow_view": true,
          "allow_fine_tuning": false,
          "organization": "*",
          "group": null,
          "is_blocking": false
        }
      ]
    }
  ]
}

__как проверять работу сервиса__
__не забываем менять порты - если БЕЗ докера, то 8080, если С докером - 8000__

пока выбрала вот этот вариант промпта и параметров 


curl http://localhost:8080/v1/chat/completions \
-H "Content-Type: application/json" \
-H "Authorization: Bearer EMPTY" \
-d '{
  "model": "Vikhr-7B-instruct_0.4_lora_r32_medium_prompt",
  "max_tokens": 400,
  "temperature": 0.1,
  "top_p": 0.95,
  "repetition_penalty": 1.1,
  "stop": ["\n\n"],
  "messages": [
    {
      "role": "system",
      "content": "Ты помощник, который анализирует отзывы на товары и выделяет атрибуты. Ниже представлен список отзывов на мандарины:\nОчень вкусные, сладкие! Никакие абхазские не нужны))\nОгромные, почти безвкусные\nХорошие мандарины, в меру сладкие, без косточек\nсладкие, косточек не попалось, кожура тонкая и легко чистится\nиз этого списка отзывов выделяем атрибуты:\nвкус: сладкий, безвкусный\nразмер: огромный\nструктура: без косточек\nкожура: тонкая и легко чистится\n\nТеперь проанализируй отзывы ниже выдели атрибуты аналогично."
    },
    {
      "role": "user",
      "content": "Огромные и не очень вкусные Замечательные!! Мне очень понравились яблоки этого сорта. Сочные, сладкие! Вата, а не яблоки Сочные, вкусные Привезли очень крупные яблоки! На вкус ароматные и сладкие. Для тех кто любит мягкие. Сочные сладкие крепкие Яблоки просто восторг!- не ожидала! Сочные, ароматные, не жесткие- даже кожа хорошо жуётся! Сладкие. Брала со скидкой. Закажу ещё на завтра. Не уверена что привезут такие же- иначе бы кг 3 заказала. Привезли в заказе ужасные яблоки!!Есть такие невозможно (( Небольшие сочные хрустящие яблочки, блеск Крупненькие оказались, ожидала мелкие яблочки. Красивее, чем на фото.) Мне для пирога. Спасибо. Впервые в продаже такие красивые сезонные яблоки. И вкусные. Яблоки вкусные, но очень большие. Хотелось бы поменьше размером. Крупные. Мало косточек. Кисло сладкие. Мякоть мягкая и сочная. Кожица не толстая. Очень вкусные! Ужасные яблоки, кожура потемневшая, внутри ватные Очень вкусные и сладкие яблоки. Сочные. Приятные яблочки. Сочные, в меру сладкие. Кожица несколько плотновата. Хорошие крупные яблоки. Вкусные. Очень ароматные и сладкие! Те, что темно-красные даже очень сладкие! Хорошие мелкие сезонные яблоки. Адекватная цена. Вкусные яблоки, душистые"
    }
  ]
}'


## выбираем промпты 

создала башевые скрипты для проверки нескольких вариантов промптов и параметров test_vikhr.sh и test_vikhr2.sh (он забирает отзывы из отдельной папки, можно тестировать несколько товаров сразу)

в консоли 
chmod +x test_vikhr.sh
./test_vikhr.sh

ниже код для парсинга всех джсонов, в которые сложились результаты

In [70]:
import os
import json
import re
import pandas as pd

RESULTS_DIR = '/home/d-mishina/vllm/test_results'
output_records = []


param_pattern = re.compile(r'tokens([0-9.]+)_temp([0-9.]+)_top([0-9.]+)_pen([0-9.]+)_')


for filename in os.listdir(RESULTS_DIR):
    if filename.endswith('.json'):
        file_path = os.path.join(RESULTS_DIR, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            try:
                data = json.load(f)
            except json.JSONDecodeError:
                print(f"Ошибка декодирования JSON в файле: {filename}")
                continue

        
        param_match = param_pattern.search(filename)
        if param_match:
            tokens, temp, top_p, penalty = param_match.groups()
        else:
            tokens, temp, top_p, penalty = None, None, None, None
            print(f"Не удалось распарсить параметры из имени файла: {filename}")


        try:
            content = data['choices'][0]['message'].get('content', '')
        except (KeyError, IndexError, AttributeError):
            print(f"Ошибка чтения содержимого в файле: {filename}")
            content = ""

        output_records.append({
            'filename': filename,
            'max_tokens': tokens,
            'temperature': temp,
            'top_p': top_p,
            'repetition_penalty': penalty,
            'full_output': content
        })


df = pd.DataFrame(output_records)


df.to_csv('analysis_summary_Vikhr-7B-instruct_0.4_lora_r32_medium_prompt.csv', index=False, encoding='utf-8-sig')
print(f"Обработано файлов: {len(output_records)}")
print("DONE!")

Обработано файлов: 2400
DONE!


In [77]:
df.head()

,filename,max_tokens,temperature,top_p,repetition_penalty,full_output
0,test1_test_cucumbers_tokens300_temp0.2_top0.8_...,300,0.2,0.8,1.3,"Неприятный привкус во вкусе, больше покупать н..."
1,test4_test_cucumbers_tokens200_temp0.05_top0.8...,200,0.05,0.85,1.3,"Сладкие, крупные, без косточек\nОтличный товар..."
2,test3_test_bananas_tokens400_temp0.1_top0.85_p...,400,0.1,0.85,1.4,""" компании, ослемы,, выморами, благодаря, "",, ..."
3,test4_test_bananas_tokens300_temp0.05_top0.85_...,300,0.05,0.85,1.1,",,,, в группе,,\n,,"
4,test5_test_bananas_tokens300_temp0.05_top0.9_p...,300,0.05,0.9,1.2,",,,, в группе,\n, в компании,"
